In [1]:
import numpy as np
import math
import numpy.polynomial.polynomial as poly
import matplotlib.pyplot as plt
from odlib import *

In [2]:
def formatInputInfo(file:str):
    info=np.loadtxt(file,dtype=str,delimiter=",")
    time=np.array([float(info[i,0]) for i in range(1,len(info))])
    timestamps=np.copy(info[1:,1])
    return time,np.array([([float(info[i][2]),float(info[i][3]),float(info[i][4])], 
                    [float(info[i][5]),float(info[i][6]),float(info[i][7])]) for i in range(1,len(info))]), timestamps

def formatResultInfo():
    info=np.loadtxt("/home/soonali/Documents/ODCode/TestResults.txt",dtype=str,delimiter=",")
    return np.array([[float(info[i][j]) for j in range(2,14)] for i in range(1,len(info))])

In [19]:
class OD:
    '''Class that performs all orbital determination calculations'''
    
    def __init__(self, inputFile:str, sunPosFile:str):
        """ Initializes OD class
            Args:
                file (str): input file name
            Returns:
                None
        """
        # constants
        self.k = 0.0172020989484 #Gaussian gravitational constant  
        self.cAU = 173.144643267 #speed of light in au/(mean solar)day  
        self.mu = 1
        self.eps = np.radians(23.4374) #Earth's obliquity
        self.toGaussian=365.2568983
        self.mu = 1
        
        self.data=Data() # Data object
        self.inputFile=inputFile
        self.sunPosFile=sunPosFile
        
    def genElements(self, pos:list, vel:list, time:float, update:bool=True):
        """ Calculates and returns the orbital elements given position, velocity, time
            Args:
                pos (list): the position vector
                vel (list): the velocity vector
                time (float): the time in Julian days
                update (bool): if True keeps the newly calculated orbital elements
            Returns:
                floats: the orbital elements; a,e,i,o,v,w,T,M
        """
        if update:
            self.pos,self.vel,self.time=pos,vel,time
            self.od=ODElements(self.pos,self.vel,self.time)
            self.a,self.e,self.i,self.o,self.v,self.w,self.T,self.M = self.od.getElements()
            return self.getElements()
        else:
            od=ODElements(pos,vel,time)
            return od.getElements()
    
    def getElements(self):
        """ Returns the orbital elements (already calculated)
            Args:
                rad (bool): True if return in radians
            Returns:
                floats: a,e,i,o,v,w,T,M
        """
        return self.a, self.e, self.i, self.o, self.v, self.w, self.T, self.M
    
    def printODElementsErr(self, file:str, date:str):
        """ Prints the OD elements and compares them to results (expected, calculated, % error)
            Args:
                file (str): actual values file name
                date (str): date that the OD elements were calculated for
            Returns:
                None
        """
        self.od.printError(self.data.getODActualData(file, date))
    
    def getT(self)->float:
        """ Returns the time of perihelion
            Args:
                None
            Returns:
                float: time of perihelion
        """
        return self.T # time of perihelion
    
    def SEL(self, taus:list, sunR2:list, rhohat2:list, coefD:list):
        """ Scalar Equation of Lagrange to calculate the roots (r) and rhos corresponding to each r
            Args:
                taus (list): a list of floats of taus [T1,T3,T]
                sunR2 (list): a list representing the sun vector R2
                rhohat2 (list): a list representing the rhohat2 vector
                coefD (list): a list of D coefficients [D0,D21,D22,D23]
            Returns:
                lists: roots (r's), rhos
        """
        T1,T3,T=taus[0],taus[1],taus[2]
        D0,D21,D22,D23=coefD[0],coefD[1],coefD[2],coefD[3]
        A1=T3/T
        B1=A1/6*(T**2-T3**2)
        A3=-T1/T
        B3=A3/6*(T**2-T1**2)
        A=(A1*D21-D22+A3*D23)/(-D0)
        B=(B1*D21+B3*D23)/(-D0)
        
        E=-2*(dot(rhohat2, sunR2))
        F=dot(sunR2,sunR2)
        
        a=-(A**2+A*E+F)
        b=-self.mu*(2*A*B+B*E)
        c=-self.mu**2*B**2
        
        coef=[c,0,0,b,0,0,a,0,1]
        res=poly.polyroots(coef)

        temproots=[]
        for val in res: 
            if np.isreal(val) and np.real(val)>0: temproots.append(np.real(val))

        temprhos=[A+B/temproots[i]**3 for i in range(len(temproots))]
        
        # ignore pairs where rho magnitude is negative
        roots=[]
        rhos=[]
        #print(temproots, temprhos)
        for i in range(len(temproots)):
            if temprhos[i]>0.0:
                roots.append(temproots[i])
                rhos.append(temprhos[i])
        
        return roots,rhos
    
    def ephemeris(self, time:float, date:str, sunFile:str):
        """ Calculates RA and Dec given time and date, using previously calculated orbital elements
            Args:
                time (float): time to determine ephemeris for in Julian Days
                date (str): date for which to determine ephemeris
                sunFile (str): file name for sun positions
            Returns:
                floats: ra, dec
        """
        n=self.k*math.sqrt(self.mu/(self.a**3))
        M=n*(time-self.T)
        E=newton(lambda E:M - E + self.e*np.sin(E), lambda E: -1+self.e*np.cos(E), M, 1e-14)
        
        pos=np.array([self.a*math.cos(E)-self.a*self.e, self.a*math.sqrt(1-self.e**2)*math.sin(E), 0])
        
        # the four rotations
        pos=rotZX(pos,np.deg2rad(self.w),np.deg2rad(self.i))
        pos=rotZX(pos,np.deg2rad(self.o),self.eps)
        
        R=self.data.getSunPos(date, sunFile)
        if np.array_equal(R, np.array([0,0,0])): raise Exception("Sun Position Not Found in SunPos.txt")
        rho=pos+R
        rhohat=rho/getMag(rho)
        
        dec=math.asin(rhohat[2])
        cra=rhohat[0]/math.cos(dec)
        sra=rhohat[1]/math.cos(dec)

        ra=getAngle(sra,cra)
        
        dec=np.rad2deg(dec)
        ra=np.rad2deg(ra)
        
        dec=DECdecimalToDMS(dec)
        ra=RAdecimalToHMS(ra)
        
        return ra,dec
        
        
    def fg(self, tau:float, r2mag:float, r2dot:list, order:int, r2:list=[]):
        """ Gets the f and g values given one time
            Args:
                tau (float): the time in Julian Days
                r2mag (float): the magnitude of r2
                r2dot (list): the velocity vector 2
                order (int): order of f and g taylor series approximations
                r2 (list): optional parameter, the position vector 2
            Returns:
                floats: f, g
        """
        if len(r2)==0: u=self.mu/r2mag**3
        else: u=self.mu/getMag(r2)**3
        
        f=1-1/2*u*tau**2
        g=tau
        
        if order>=3:
            z=dot(r2,r2dot)/(dot(r2,r2))
            q=dot(r2dot,r2dot)/(dot(r2,r2))-u
            f+=1/2*u*z*tau**3
            g+=-1/6*u*tau**3
        
        if order>=4:
            f+=1/24*(3*u*q-15*u*z**2+u**2)*tau**4
            g+=1/4*u*z*tau**4
        
        return f, g
        
    def getFGVals(self, tau1:float, tau3:float, r2mag:float, r2dot:list, order1:int, order2:int, r2:list=[]):
        """ Gets all f and g values
            Args:
                tau1 (float): the time in Julian Days for observation 1 from obs 2(T1-T2)
                tau3 (float): the time in Julian days for observation 3 from obs 2(T3-T2)
                r2mag (float): the magnitude of r2
                r2dot (list): the velocity vector 2
                order1 (int): Order of Taylor Series expansion for f and g values for observation 1
                order2 (int): Order of Taylor Series expansion for f and g values for observation 3
                r2 (list): optional parameter, the position vector 2
            Returns:
                lists: [f1,f3], [g1,g3]
        """
        f1,g1=self.fg(tau1,r2mag,r2dot,order1,r2)
        f3,g3=self.fg(tau3,r2mag,r2dot,order2,r2)
        return [f1,f3], [g1,g3]
    
    def getDCoef(self, ra:list, dec:list, R1:list, R2:list, R3:list):
        """ Gets the D coefficients given the ra and dec for three observations (in radians)
            Args:
                ra (list): the right ascensions for three observations (radians)
                dec (list): the declinations for three observations (radians)
                R1 (list): the sun vector for observation 1
                R2 (list): the sun vector for observation 2
                R3 (list): the sun vector for observation 3
            Returns:
                list: [D0,D11,D12,D13,D21,D22,D23,D31,D32,D33], [rhohat1, rhohat2, rhohat3]
        """
        ra1,ra2,ra3=ra[0],ra[1],ra[2]
        dec1,dec2,dec3=dec[0],dec[1],dec[2]
        
        rhohat1=np.array([np.cos(ra1)*np.cos(dec1), np.sin(ra1)*np.cos(dec1), np.sin(dec1)])
        rhohat2=np.array([np.cos(ra2)*np.cos(dec2), np.sin(ra2)*np.cos(dec2), np.sin(dec2)])
        rhohat3=np.array([np.cos(ra3)*np.cos(dec3), np.sin(ra3)*np.cos(dec3), np.sin(dec3)])
        
        D0=dot(rhohat1, cross(rhohat2,rhohat3))
        D11=dot(cross(R1, rhohat2),rhohat3)
        D12=dot(cross(R2, rhohat2),rhohat3)
        D13=dot(cross(R3, rhohat2),rhohat3)
        
        D21=dot(cross(rhohat1,R1), rhohat3)
        D22=dot(cross(rhohat1,R2), rhohat3)
        D23=dot(cross(rhohat1,R3), rhohat3)
        
        D31=dot(rhohat1, cross(rhohat2,R1))
        D32=dot(rhohat1, cross(rhohat2,R2))
        D33=dot(rhohat1, cross(rhohat2,R3))
     
        return [D0,D11,D12,D13,D21,D22,D23,D31,D32,D33], np.array([rhohat1, rhohat2, rhohat3])
    
    def MoGGenData(self,selTime:list=[],selDate:list=[]):
        self.data.getInput(self.inputFile,self.sunPosFile) # formats and generates info
        # generate data
        if not(selTime==[]): # using Julian day times
            ra,dec=[],[]
            for time in selTime:
                r,d=self.data.getRADECInput(time=time)
                ra.append(np.deg2rad(r))
                dec.append(np.deg2rad(d))
            R1=self.data.getSunInput(time=selTime[0])
            R2=self.data.getSunInput(time=selTime[1])
            R3=self.data.getSunInput(time=selTime[2])
            # calculate the taus
            t1,t2,t3=selTime[0],selTime[1],selTime[2]
            ts=[t1,t2,t3]
            T1=t1-t2
            T3=t3-t2
            T=t3-t1
            taus = [T1,T3,T] # in Gaussian days
                
        elif not(selDate==[]):
            ra,dec=[],[]
            for date in selDate:
                r,d=self.data.getRADECInput(date=date)
                ra.append(np.deg2rad(r))
                dec.append(np.deg2rad(d))
            R1=self.data.getSunInput(date=selDate[0])
            R2=self.data.getSunInput(date=selDate[1])
            R3=self.data.getSunInput(date=selDate[2])
            # calculate the taus given the dates
            t1,t2,t3=self.data.getJDTime(selDate[0]),self.data.getJDTime(selDate[1]),self.data.getJDTime(selDate[2])
            ts=[t1,t2,t3]
            T1=t1-t2
            T3=t3-t2
            T=t3-t1
            taus = [T1*self.k,T3*self.k,T*self.k]
            
        else: raise Exception("No data given")
        
        return ra,dec,np.array(R1),np.array(R2),np.array(R3),taus,np.array(ts)
    
    def MoGCalcRhos(self, rhohats:list, coefD:list, fvals:list, gvals:list):
        f1,f3=fvals
        g1,g3=gvals
        D0,D11,D12,D13,D21,D22,D23,D31,D32,D33=coefD
        C1=g3/(f1*g3-g1*f3)
        C2=-1
        C3=-g1/(f1*g3-g1*f3)
        rho1=(C1*D11+C2*D12+C3*D13)/(C1*D0)
        rho2=(C1*D21+C2*D22+C3*D23)/(C2*D0)
        rho3=(C1*D31+C2*D32+C3*D33)/(C3*D0)
        rhomags=np.array([rho1,rho2,rho3])
        rhos=np.array([rhohats[0]*rho1, rhohats[1]*rho2, rhohats[2]*rho3])
        return rhos, rhomags
    
    def MoGCalcPos(self, rhos:list, Rs:list):            
        return rhos-Rs
        
    def MoGGetErr(self, prev:list, cur:list, tolerance:float)->bool:
        for i in range(len(prev)):
            if abs(prev[i]-cur[i])>tolerance: return False
        return True
    
    def MoGGetAdjustedTaus(self, origt:list, rhomags:float)->list:
        ts=np.copy(origt)-rhomags/self.cAU
        t1,t2,t3=ts
        return [(t1-t2)*self.k,(t3-t2)*self.k,(t3-t1)*self.k]
    
    def MoG(self,selTime:list=[],selDate:list=[]):
        # generate data
        ra,dec,R1,R2,R3,taus,ts=self.MoGGenData(selTime,selDate)
        
        # calculate the initial estimate for r2 DONE
        coefD,rhohats=self.getDCoef(ra, dec, R1, R2, R3)

        SELcoefD=[coefD[0],coefD[4],coefD[5],coefD[6]] # [D0,D11,D12,D13,D21,D22,D23,D31,D32,D33] -> [D0,D21,D22,D23]
        r2MagGuesses,rhoMagGuesses=self.SEL(taus, R2, rhohats[1], SELcoefD)
        
        results=[]
        
        # calculate the f and g values to second order
        for r2mag in r2MagGuesses:
            # initial guesses
            fvals,gvals=self.getFGVals(taus[0], taus[1], r2mag, [], 2, 2) # using r2mag
            rhos,rhomags=self.MoGCalcRhos(rhohats, coefD, fvals, gvals)
            rs=self.MoGCalcPos(rhos, np.array([R1,R2,R3]))
   
            # calculate the central velocity vector
            f1,f3=fvals
            g1,g3=gvals
            r2dot = (-f3/(-f3*g1+g3*f1))*rs[0] + (f1/(f1*g3-f3*g1))*rs[2]
            print(rs)
            counter=0
            timedOut=True
            
            while counter<10000: # timeout
                prev=rs[1]
                
                # time adjustment
                newtaus=self.MoGGetAdjustedTaus(ts, rhomags)
                
                fvals,gvals=self.getFGVals(newtaus[0], newtaus[1], r2mag, r2dot, 4, 4, rs[1]) # using r2mag
                rhos,rhomags=self.MoGCalcRhos(rhohats, coefD, fvals, gvals)
                rs=self.MoGCalcPos(rhos, np.array([R1,R2,R3]))
                
                f1,f3=fvals
                g1,g3=gvals
                r2dot = (-f3/(-f3*g1+g3*f1))*rs[0] + (f1/(f1*g3-f3*g1))*rs[2]
                
                counter+=1
                
                if (self.MoGGetErr(prev, rs[1], 1e-23)): 
                    timedOut=False
                    break
            
            if timedOut: print("Timed out")
            else: 
                # rotate around x axis to get to ecliptic
                r2=rotX(np.array(rs[1]),-self.eps)
                r2dot=rotX(r2dot,-self.eps)
                results.append([r2, r2dot]) # position and velocity
            
        
        
        for result in results:
            pos,vel=result
            #print(pos,np.array(vel)*(2*math.pi)/365.2568983,ts[1])
            print(self.genElements(pos,np.array(vel)*(2*math.pi)/365.2568983,ts[1]))
            a,e,i,o,w,m,t= 1.535501123505049E+00, 1.980131722243141E-01, 1.127908127242095E+01, 1.961976310756240E+02, 1.425325538303718E+02, 3.163937598615976E+02, 2459856.859881772194


    
            self.od.printError([e,0,i,o,w,t,0,m,100,a])

        
        

    

In [21]:
file2 = "/home/soonali/Desktop/SSP2022/TestInput.txt"
file = "/home/soonali/Desktop/SSP2022/SoongInput.txt"
sunFile="/home/soonali/Desktop/SSP2022/SunPos.txt"
date = '2018-Jul-14 00:00:00'
#jdtime = 2458313.5000000    
data=Data()
od=OD(file,sunFile)
#pos,vel,time=data.getTestInput(file2, date)
#od.genElements(pos,vel,time)
#od.MoG(selDate=['2021 06 25 00:00:00.000','2021 07 05 00:00:00.000','2021 07 15 00:00:00.000'])
od.MoG(selDate=['2022-06-28 04:33:44.089','2022-07-12 04:16:40.826','2022-07-14 04:41:39.025'])

[[-0.05164793 -1.312605   -0.31448548]
 [ 0.1528297  -1.29339186 -0.28950336]
 [ 0.18201409 -1.28804048 -0.28532289]]
(1.5347326380022122, 0.1970528139135287, 11.297805611270112, 196.30891625203841, 297.6085808028801, 142.4976525114411, 2459162.6487638657, 316.2319950352826)
Semi-major axis: 1.535501123505049 1.5347326380022122 0.050047863272331704
Eccentricity: 0.1980131722243141 0.1970528139135287 0.48499718478198134
Inclination: 11.27908127242095 11.297805611270112 0.16600943283338615
Longitude of Ascending Node: 196.197631075624 196.30891625203841 0.05672095825230317
True anomaly: 100 297.6085808028801 197.6085808028801
Argument of perihelion: 142.5325538303718 142.4976525114411 0.024486559731640926
Time of Perihelion Passage T: 2459856.859881772 2459162.6487638657 0.028221606274271654
Mean Anomaly: 316.3937598615976 316.2319950352826 0.05112769176792712


In [22]:
file = "/home/soonali/Desktop/SSP2022/TestInput.txt"
sunFile="/home/soonali/Desktop/SSP2022/SunPos.txt"
date = '2018-Jul-14 00:00:00'
#jdtime = 2458313.5000000    
data=Data()
pos,vel,time=data.getTestInput(file, date)
od=OD(file,sunFile)
od.genElements(pos,vel,time)

# testing OD elements class
def testODElements(od):
    actual="/home/soonali/Desktop/SSP2022/TestResults.txt"
    od.printODElementsErr(actual, date)
    
testODElements(od)

Semi-major axis: 1.056800055744403 1.0568000556596946 8.015561884787836e-09
Eccentricity: 0.3442331093278357 0.3442331094157307 2.5533576582325764e-08
Inclination: 25.1552516656358 25.155251665637707 7.584131023113933e-12
Longitude of Ascending Node: 236.2379806531768 236.23798065316336 5.690646577409404e-12
True anomaly: 158.9559248709949 158.9559248763771 3.385982069686107e-09
Argument of perihelion: 255.5046142809498 255.5046142714118 3.732998971925637e-09
Time of Perihelion Passage T: 2458158.720849546 2458158.720849547 3.7886999188303465e-14
Mean Anomaly: 140.4194576216498 140.41945762513234 2.4800982265579533e-09


In [ ]:
def testEphemeris(od):
    newdate = '2018-Aug-03 00:00:00'
    newjdtime = 2458333.5000000
    sunFile="/home/soonali/Desktop/SSP2022/SunPos.txt"
    ra,dec=od.ephemeris(newjdtime, newdate, sunFile)
    
    # for jul 14 00:00:00
    #ra,dec=od.ephemeris(jdtime, date)
    #print("\n% error ra:", error(HMStoDeg(18,14,30.53),HMStoDeg(ra[0],ra[1],ra[2])))
    #print("% error dec:", error(DMStoDeg(36,9,46.0),DMStoDeg(dec[0],dec[1],dec[2])))

    #print("\nexpected:")
    #print("ra: 18 14 30.53")
    #print("dec: +36 09 46.0")
    
    print("calculated:")
    print("ra:", ra)
    print("dec:",dec)
    
    print("\n% error ra:", error(HMStoDeg(17,42,21.12),HMStoDeg(ra[0],ra[1],ra[2])))
    print("% error dec:", error(DMStoDeg(31,52,26.8),DMStoDeg(dec[0],dec[1],dec[2])))

    print("\nexpected:")
    print("ra: 17 42 21.12")
    print("dec: +31 52 26.8")
    
testEphemeris(od)

In [ ]:
def testFG(od):
    tau1 = -0.32618617484601165  
    tau3 = 0.0508408854033231  
    r2 = [0.26799552002875776, -1.3726277901924608, -0.5026729612047128]  
    r2dot = [0.8456809141954584, -0.3838382184712308, 0.14215854191172816] 

    f,g=od.getFGVals(tau1,tau3,0,r2dot,3,3,r2)
    f1,f3=f[0],f[1]
    g1,g3=g[0],g[1]

    print("Test case 2:")
    print("f1:",f1,error(0.9821596284506794,f1))
    print("f3:",f3,error(0.9996124342607986,f3))
    print("g1:",g1,error(-0.32442392608030396,g1))
    print("g3:",g3,error(0.05083421257607972,g3))

    tau1 = -0.3261857571141891  
    tau3 = 0.05084081855693949  
    r2 = [0.26662393644794813, -1.381475976476564, -0.5048589337503169]  
    r2dot = [0.8442117090940343, -0.39728396707075087, 0.14202728258915864]  

    f,g=od.getFGVals(tau1,tau3,0,r2dot,4,4,r2)
    f1,f3=f[0],f[1]
    g1,g3=g[0],g[1]

    print("\nTest case 3:")
    print("f1:",f1,error(0.9823149707782799  ,f1))
    print("f3:",f3,error(0.99961917185922  ,f3))
    print("g1:",g1,error(-0.32418770657924106  ,g1))
    print("g3:",g3,error(0.05083441832100904,g3))
    
    r2dot=[]
    f,g=od.getFGVals(tau1,tau3,0,r2dot,2,2,r2)
    f1,f3=f[0],f[1]
    g1,g3=g[0],g[1]

    print("\nTest case 3:")
    print("f1:",f1,error(0.9823149707782799  ,f1))
    print("f3:",f3,error(0.99961917185922  ,f3))
    print("g1:",g1,error(-0.32418770657924106  ,g1))
    print("g3:",g3,error(0.05083441832100904,g3))
    
    
testFG(od)


In [ ]:
n=np.array([])
n.shape